In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split 
from sklearn.datasets import make_blobs 
from sklearn import svm

from h import *
from W_unitary import *
from U_unitary import *
from qiskit.algorithms.optimizers import COBYLA, ADAM, SPSA, SLSQP, POWELL, L_BFGS_B, TNC, AQGD

%matplotlib inline
#plt.rcParams['figure.figsize'] = (10,6)
#plt.rcParams['figure.dpi'] = 100
#sns.set()

#%matplotlib inline
#%load_ext autoreload
#%autoreload 2

<ipython-input-1-9b0f1830d79c>:14: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms.optimizers import COBYLA, ADAM, SPSA, SLSQP, POWELL, L_BFGS_B, TNC, AQGD


In [2]:
def MSE_loss(theta, X_train, y_train):
    
    # new_y_train = []
    # for i in range(0, len(y_train)):
    #     if y_train[i] == 0:
    #         new_y_train.append(-1)
    #     else:
    #         new_y_train.append(1)
    
    predictions = []
    for i in range(0, len(y_train)):
        predictions.append(h(X_train[i], theta, 2, 2, shots))
    
    error = []
    for i in range(0, len(predictions)):
        parity = predictions[i] - y_train[i]
        error.append(parity)

    norm = np.linalg.norm(error)
    return norm


def k_fold_split(X, y, ele_per_split, i):
    
    X_train =  np.concatenate( (X[:ele_per_split*i, :], X[ele_per_split*(i+1):, :]) )
    X_test = X[ele_per_split*i:ele_per_split*(i+1), :]
    
    y_train = np.concatenate( (y[:ele_per_split*i], y[ele_per_split*(i+1):]) )
    y_test = y[ele_per_split*i:ele_per_split*(i+1)]
    
    return X_train, X_test, 2*y_train-1, 2*y_test-1

In [3]:
RANDOM_STATE=42
seed = np.random.seed(RANDOM_STATE)

n = 2
d = 2
theta = 2*np.pi*np.random.random(n*d*3)

n_samples = 100
noise = 0.3
k = 4
if n_samples%k != 0:
    print("Prefered to have k as a divisor of n_samples")
ele_per_split = n_samples//k

X, y = make_moons(n_samples, noise=noise)

shots = 1024

In [5]:
thetas = np.zeros((len(theta), k))

for i in tqdm(range(k)):
    
    X_train, X_test, y_train, y_test = k_fold_split(X, y, ele_per_split, i)

    objective_function = lambda theta: MSE_loss(theta, X_train, y_train)

    optimizer = COBYLA(maxiter=100)
    theta_opt = optimizer.minimize(objective_function, theta).x

    thetas[:, i] = theta_opt

    y_optimizer_train = np.zeros(len(y_train))
    for j in range(0, len(y_train)):
        y_optimizer_train[j] = ( h(X_train[j], theta_opt, n, d, shots) )

    y_optimizer_test = np.zeros(len(y_test))
    for j in range(0, len(y_test)):
        y_optimizer_test[j] = ( h(X_test[j], theta_opt, n, d, shots) )
    
    #training_accuracy = (int(1/2*np.sum(abs(y_optimizer_train-y_train))) / n_samples - n_samples//k) * 100
    #testing_accuracy = (1/2*np.sum(abs(y_optimizer_test-y_test)) / n_samples//k) * 100
    print("Split: ", i+1)
    print("Optimal parameters: ", theta_opt)
    print("Training accuracy: ", int(1/2*np.sum(abs(y_optimizer_train-y_train))), "incorrectly out of ", n_samples - n_samples//k)
    print("Testing accuracy: ", 1/2*np.sum(abs(y_optimizer_test-y_test)), "incorrectly out of ", n_samples//k)

 25%|█████████████████████                                                               | 1/4 [00:59<02:58, 59.65s/it]

Split:  1
Optimal parameters:  [2.18635173 5.91048939 4.284906   3.59462312 0.5242556  0.66558844
 0.24681434 4.72776911 3.46253916 4.28224945 0.1777993  5.77942507]
Training accuracy:  29 incorrectly out of  75
Testing accuracy:  13.0 incorrectly out of  25


 50%|██████████████████████████████████████████                                          | 2/4 [02:04<02:05, 62.59s/it]

Split:  2
Optimal parameters:  [3.17887253 7.16681958 4.58984355 3.67524516 0.86966649 1.04895217
 0.77480851 6.13784075 3.77303594 4.44507015 0.12545513 6.09412333]
Training accuracy:  29 incorrectly out of  75
Testing accuracy:  10.0 incorrectly out of  25


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [03:05<01:01, 61.85s/it]

Split:  3
Optimal parameters:  [3.35332145 6.97351461 4.59920414 3.76147939 0.98028306 0.98010904
 0.36494313 5.44240343 3.77696203 4.44897234 0.12932479 6.09412333]
Training accuracy:  32 incorrectly out of  75
Testing accuracy:  13.0 incorrectly out of  25


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [04:06<00:00, 61.65s/it]

Split:  4
Optimal parameters:  [ 2.35330497  5.97351416  4.59925358  3.76148219  2.34701663  0.61341988
  2.11400598  5.06514168  3.02786113  4.18094998 -0.98644229  6.09412333]
Training accuracy:  30 incorrectly out of  75
Testing accuracy:  15.0 incorrectly out of  25


In [6]:
theta_opt_avg = np.mean(thetas, axis=1)
X_val, y_val = make_moons(n_samples//k, noise=noise)

y_val = 2*y_val-1
y_optimizer_val = np.zeros(len(y_val))
for j in tqdm(range(0, len(y_val))):
    y_optimizer_val[j] = ( h(X_val[j], theta_opt, n, d, shots) )

#validation_accuracy = (1/2*np.sum(abs(y_optimizer_val-y_val)) / n_samples//k) * 100
print("Optimal average parameters: ", theta_opt_avg)
print("Validation accuracy: ", 1/2*np.sum(abs(y_optimizer_val-y_val)), "incorrectly out of ", n_samples//k)

#print(sum(y_val!=y_optimizer_val))
# print(y_val)
# print(y_optimizer_val)
# print(y_val==y_optimizer_val)

100%|█████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 103.32it/s]

Optimal average parameters:  [ 2.76796267  6.50608443  4.51830182  3.69820746  1.18030544  0.82701738
  0.87514299  5.34328874  3.51009956  4.33931048 -0.13846577  6.01544877]
Validation accuracy:  11.0 incorrectly out of  25
